<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ASLG_and_Daily_Merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.8 MB/s eta 0:00:00


In [2]:
import spacy
from datasets import load_dataset

# Load the Hugging Face dataset
dataset = load_dataset("HamdanXI/cleaned_daily_dialog_sentence")

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Function to extract nouns, verbs, adjectives, and adverbs
def extract_relevant_tokens(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']])

# Apply the function to the dataset
dataset = dataset.map(lambda example: {'nouns_verbs_adj_adv_sentence': extract_relevant_tokens(example['dialogue'])})

# Rename columns
dataset = dataset.rename_column("dialogue", "text")
dataset = dataset.rename_column("nouns_verbs_adj_adv_sentence", "gloss")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/77350 [00:00<?, ? examples/s]

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

In [3]:
def remove_short_words_function(example):
    words = example['gloss'].split()
    filtered_words = [word for word in words if len(word) > 1]
    example['gloss'] = ' '.join(filtered_words)
    return example

# Remove 2 words or less gloss sentence length
def filter_function(example):
    return len(example['gloss'].split()) >= 2

# All to capital letters
def capitalize_function(example):
    example['gloss'] = example['gloss'].upper()
    return example

# Apply the filter function to each split of the dataset
filtered_dataset = dataset.map(remove_short_words_function)
filtered_dataset = filtered_dataset.filter(filter_function)
filtered_dataset = filtered_dataset.map(capitalize_function)

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

Filter:   0%|          | 0/77350 [00:00<?, ? examples/s]

Map:   0%|          | 0/71766 [00:00<?, ? examples/s]

In [4]:
print(filtered_dataset)
print(filtered_dataset['train'][0:5])

DatasetDict({
    train: Dataset({
        features: ['text', 'gloss'],
        num_rows: 71766
    })
})
{'text': ['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .', 'What do you mean ? It will help us to relax .', "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?", "I guess you are right.But what shall we do ? I don't feel like sitting at home ."], 'gloss': ['SAY GOING FEW BEERS DINNER', 'KNOW TEMPTING REALLY GOOD FITNESS', 'MEAN HELP RELAX', 'REALLY THINK SO DO JUST MAKE FAT ACT SILLY REMEMBER LAST TIME', 'GUESS RIGHT DO FEEL SITTING HOME']}


In [5]:
from datasets import load_dataset, concatenate_datasets, Dataset

aslg_dataset = load_dataset("aslg_pc12")

Generating train split:   0%|          | 0/87710 [00:00<?, ? examples/s]

In [6]:
filtered_dataset['train'] = filtered_dataset['train'].map(lambda examples: {'gloss': examples['gloss'], 'text': examples['text']})
merged_dataset = concatenate_datasets([aslg_dataset['train'], filtered_dataset['train']])

Map:   0%|          | 0/71766 [00:00<?, ? examples/s]

In [7]:
import pandas as pd

# Convert Dataset to pandas DataFrame
df = merged_dataset.to_pandas()

# Remove duplicates based on 'text' column using pandas
df_unique = df.drop_duplicates(subset='text')

# Convert back to a Dataset object
unique_merged_dataset = Dataset.from_pandas(df_unique)

# To see the size reduction after removing duplicates
print(f"Number of rows before removing duplicates: {len(merged_dataset)}")
print(f"Number of rows after removing duplicates: {len(unique_merged_dataset)}")

Number of rows before removing duplicates: 159476
Number of rows after removing duplicates: 144285


In [8]:
unique_merged_dataset

Dataset({
    features: ['gloss', 'text', '__index_level_0__'],
    num_rows: 144285
})

In [9]:
unique_merged_dataset = unique_merged_dataset.remove_columns(['__index_level_0__'])

In [ ]:
!huggingface-cli login

In [11]:
unique_merged_dataset.push_to_hub("gloss_merged_dataset_with_adj_adv")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/145 [00:00<?, ?ba/s]